In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = "3"
import torch
import pandas as pd
import numpy as np
from time import time 

from fastai import *
from fastai.basic_data import *
from fastai.basic_data import *
from fastai.tabular import *
from fastai.basic_data import DataBunch
from fastai.tabular import TabularModel

import cudf

import nvtabular as nvt
from nvtabular.ops import Normalize, FillMissing, Categorify, get_embedding_sizes
from nvtabular.torch_dataloader import AsyncTensorBatchDatasetItr, DLDataLoader

import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
%matplotlib inline

/opt/conda/lib/python3.8/site-packages/numba/cuda/envvars.py:17: NumbaWarning: 
Environment variables with the 'NUMBAPRO' prefix are deprecated and consequently ignored, found use of NUMBAPRO_NVVM=/usr/local/cuda/nvvm/lib64/libnvvm.so.

For more information about alternatives visit: ('http://numba.pydata.org/numba-doc/latest/cuda/overview.html', '#cudatoolkit-lookup')
  warnings.warn(errors.NumbaWarning(msg))
/opt/conda/lib/python3.8/site-packages/numba/cuda/envvars.py:17: NumbaWarning: 
Environment variables with the 'NUMBAPRO' prefix are deprecated and consequently ignored, found use of NUMBAPRO_LIBDEVICE=/usr/local/cuda/nvvm/libdevice/.

For more information about alternatives visit: ('http://numba.pydata.org/numba-doc/latest/cuda/overview.html', '#cudatoolkit-lookup')
  warnings.warn(errors.NumbaWarning(msg))


In [2]:
torch.__version__, cudf.__version__

('1.5.1', '0.15.0a+3198.g1f41adf16.dirty')

In [3]:
%load_ext snakeviz
# load snakeviz if you want to run profiling

<h3> Dataset Gathering: Define files in the training and validation datasets. </h3>

In [4]:
# data_path = '/rapids/notebooks/jperez/Documents/ds-itr/examples/'
data_path = '/datasets/outbrain/jp_out/output/'
#df_test = 'test/'
df_valid = 'validation_feature_vectors_integral.csv/'
df_train = 'train_feature_vectors_integral_eval.csv/'

train_set = [data_path + df_train + x for x in os.listdir(data_path + df_train) if x.startswith("part")][:20] 
valid_set = [data_path + df_valid + x for x in os.listdir(data_path + df_valid) if x.startswith("part")][:5] 

In [5]:
len(train_set), len(valid_set)

(20, 5)

<h4>Grab column information</h4>

In [6]:
cols = open(data_path + 'train_feature_vectors_integral_eval.csv.header').read().splitlines()

In [7]:
cat_names = ['display_id', 'is_leak', 'doc_event_id', 'ad_id', 'doc_id', 'doc_ad_entity_id', 'doc_event_entity_id', 'doc_event_entity_id', 'doc_ad_source_id', 'doc_event_source_id', 'event_geo_location', 'ad_advertiser', 'event_country_state', 'doc_ad_publisher_id', 'doc_event_publisher_id', 'doc_ad_topic_id', 'doc_event_topic_id', 'event_country', 'doc_ad_category_id', 'doc_event_category_id', 'event_hour', 'event_platform', 'traffic_source', 'event_weekend', 'user_has_already_viewed_doc']
cont_names =  ['pop_ad_id_conf', 'pop_document_id_conf', 'user_doc_ad_sim_categories_conf', 'user_doc_ad_sim_topics_conf', 'pop_publisher_id_conf', 'pop_advertiser_id_conf', 'pop_campaign_id_conf', 'pop_source_id_conf', 'pop_entity_id_conf', 'pop_topic_id_conf', 'pop_category_id_conf', 'pop_ad_id', 'pop_document_id', 'pop_publisher_id', 'pop_advertiser_id', 'pop_campaign_id', 'pop_source_id', 'pop_entity_id', 'pop_topic_id', 'pop_category_id', 'user_doc_ad_sim_categories', 'user_doc_ad_sim_topics', 'user_doc_ad_sim_entities', 'doc_event_doc_ad_sim_categories', 'doc_event_doc_ad_sim_topics', 'doc_event_doc_ad_sim_entities', 'user_views', 'ad_views', 'doc_views', 'doc_event_days_since_published', 'doc_event_hour', 'doc_ad_days_since_published'] #+ [i for i in ds.columns if i not in cat_names and i not in ['label']]
cat_names = [name for name in cat_names if name in cols]
cont_names = [name for name in cont_names if name in cols]


<h3>Preprocessing:</h3> <p>Select operations to perform, create the Preprocessor object, create dataset iterator object and collect the stats on the training dataset</p>

In [8]:
freq_threshes = {}
for x in cat_names:
    freq_threshes[x] = 1
freq_threshes

{'display_id': 1,
 'is_leak': 1,
 'doc_event_id': 1,
 'ad_id': 1,
 'doc_id': 1,
 'doc_ad_source_id': 1,
 'doc_event_source_id': 1,
 'event_geo_location': 1,
 'ad_advertiser': 1,
 'event_country_state': 1,
 'doc_ad_publisher_id': 1,
 'doc_event_publisher_id': 1,
 'event_country': 1,
 'event_hour': 1,
 'event_platform': 1,
 'traffic_source': 1,
 'event_weekend': 1,
 'user_has_already_viewed_doc': 1}

In [9]:
%%time
proc = nvt.Workflow(cat_names=cat_names, cont_names=cont_names, label_name=['label'])

CPU times: user 11 µs, sys: 2 µs, total: 13 µs
Wall time: 16 µs


In [10]:
proc.add_cont_preprocess([FillMissing(replace=True), Normalize(replace=True)])
proc.add_cat_preprocess(Categorify(replace=True, freq_threshold=1))

In [11]:
%%time
trains_itrs = nvt.Dataset(train_set,names=cols, engine='csv')
valids_itrs = nvt.Dataset(valid_set,names=cols, engine='csv')

CPU times: user 9.94 ms, sys: 132 ms, total: 142 ms
Wall time: 153 ms


In [12]:
output_path_train = './jp_outbrains/train'
output_path_valid = './jp_outbrains/valid'

In [13]:
%%time
proc.apply(trains_itrs, apply_offline=True, record_stats=True, output_path=output_path_train, shuffle=False)

ValueError: All values must be of equal length

In [14]:
%%time
proc.apply(valids_itrs, apply_offline=True, record_stats=False, output_path=output_path_valid, shuffle=False)

ValueError: All values must be of equal length

In [15]:
new_train_set = [os.path.join(output_path_train, x) for x in os.listdir(output_path_train) if x.endswith("parquet")]
new_valid_set = [os.path.join(output_path_valid, x) for x in os.listdir(output_path_valid) if x.endswith("parquet")]

<h5>Gather embeddings using statistics gathered in the Read phase.</h5>

In [16]:
embeddings = list(get_embedding_sizes(proc).values())

In [17]:
embeddings

[(3698, 16),
 (198769, 16),
 (1159426, 16),
 (753, 16),
 (5894, 16),
 (212565, 16),
 (866, 16),
 (5518, 16),
 (76717, 16),
 (213, 16),
 (974, 16),
 (1272, 16),
 (7, 5),
 (7, 5),
 (3, 3),
 (2, 2),
 (4, 3),
 (3, 3)]

<h5>Create the file iterators using the FileItrDataset Class.</h5>

In [18]:
%%time
t_batch_sets = nvt.Dataset(new_train_set, engine='parquet') 
v_batch_sets = nvt.Dataset(new_valid_set, engine='parquet')

ValueError: empty urlpath sequence

In [19]:
train_dataset = AsyncTensorBatchDatasetItr(t_batch_sets, 
                                           batch_size=100000, 
                                           cats=cat_names, 
                                           conts=cont_names, 
                                           labels=["label"])
valid_dataset = AsyncTensorBatchDatasetItr(v_batch_sets, 
                                           batch_size=100000, 
                                           cats=cat_names, 
                                           conts=cont_names, 
                                           labels=["label"])

NameError: name 't_batch_sets' is not defined

In [ ]:
# %%time
# t_chain = torch.utils.data.ChainDataset(t_batch_sets)
# v_chain = torch.utils.data.ChainDataset(v_batch_sets)

In [ ]:
# proc.columns_ctx['final']['ctx']

<h5>Use the Deep Learning Collator to create a collate function to pass to the dataloader.</h5>

In [ ]:
def gen_col(batch):
    batch = batch[0]
    return (batch[0], batch[1]), batch[2].long()

In [ ]:
%%time
t_data = DLDataLoader(train_dataset, collate_fn=gen_col, pin_memory=False, num_workers=0)
v_data = DLDataLoader(valid_dataset, collate_fn=gen_col, pin_memory=False, num_workers=0)

<h4>After creating the Dataloaders you can leverage fastai framework to create Machine Learning models</h4>

In [ ]:
databunch = DataBunch(t_data, v_data, collate_fn=dlc.gdf_col, device="cuda")

In [ ]:
%%time
model = TabularModel(emb_szs = embeddings, n_cont=len(cont_names), out_sz=2, layers=[512,256])

learn =  Learner(databunch, model, metrics=[accuracy])
learn.loss_func = torch.nn.CrossEntropyLoss()


In [ ]:
learn.lr_find()

In [ ]:
learn.recorder.plot(show_moms=True, suggestion=True)

In [ ]:
learning_rate = 2.75e-2
epochs = 1

In [ ]:
start = time()
learn.fit_one_cycle(epochs,learning_rate)
t_final = time() - start 

In [ ]:
t_final